# Microchallenge 0
## Preamble
In this year's microchallenges we play around with strings of bits. We may represent a bit string as printable text but when we do so we are using rather more bytes than strictly necessary. So we may sometimes also want to dump bit strings to files in a more compact binary format. But then, to read back the same strings from the binary file, we must have clear conventions to tell us where each bit string begins and ends. (How could you make an "end of string" marker that you could unequivocally recognise later, without disallowing any string? Hang on...)

First, **padding**. To make a bit string fit into an integer number of bytes, we may have to pad it at the end. To be able to remove the padding we must define a **convention**. Here is ours: **append a 1 to the unpadded bit string and then as many 0s as necessary to make its length in bits a multiple of 8 (and even more if you need to match a given fixed length in bytes)**. This convention has the disadvantage that bit strings whose length in bits is a multiple of 8 end up occupying an extra byte than they otherwise would, but it has the advantage of having **a simple inverse**: starting from the right end of the string, drop all consecutive zeros (if any) until you find the first 1, and then drop that 1 too. What's left is the original, unpadded string. This works for strings of any length and content, including the empty string. Nice.

Second, **splitting**. Now that you have a (padded) bit string that evenly divides into 8-bit chunks, take each chunk as a byte and read it left to right. Note that therefore **the first bit of the string is the most (not least) significant bit of the first byte**.

A couple of definitions: when we need to (which isn't very frequently), we write |A| for the length, in bits, of bit string A. And we say that bit string A is a **prefix** of bit string B if and only if the first |A| bits of B, assuming they exist, are the same as A (in other words, iff B "begins with" A).

The general problem space for all our microchallenges this year is **the conversion of a stream of 8-bit symbols (henceforth "the symbols") into a stream of bit strings of variable size ("the images"), reversibly**. The conversion is based on a lookup table that establishes a bijection between symbols and images. Of course the fixed-size symbols can themselves be represented as bit strings, so we may alternatively say that the lookup table is a bijection between a set of fixed-size bit strings and a set of variable-size bit strings.

There are many possible representations for this table, with different trade-offs between readability and compactness, but here we use the following two. (You may come up with more convenient ones and write converters for them if you wish, but do use these two standards when coding the microchallenges.)

1) **The text representation** is a text file with one line per (symbol, image) pair. Each line begins with the text representation of the symbol, which can be one of the following two options: **a single ASCII character** (if and only if all the symbols in the table are between decimal 33 and decimal 126); otherwise **a two-hex-digit representation** of the symbol. This is followed by a space and then by the corresponding variable-length bit string for the image, that is to say **a sequence of the ASCII characters "0" and "1"**. This being a text file, the line is obviously terminated by a newline. Example below. A table stored in a file in text format is denoted by the extension **.tbt** (a mnemonic for: "table, text").

2) **The binary representation** uses fixed field sizes, in bits, to store both **the symbol (one byte, unpadded)** and **the image (two bytes, padded)**. Clearly this means that images **longer than 15 bits are not representable** in this binary format; so the valid tables that this format can represent are a subset of those that the text format can represent. A table stored in a file in text format is denoted by the extension **.tbb** (a mnemonic for: "table, binary").

## Questions  
### (a)
Write a tbt2tbb program that converts a lookup table from the text representation (extension .tbt) to the binary table representation (extension .tbb). Raise an explanatory error if this cannot be done.  
  
For example, if the input file is called tomato.tbt and contains those strings:  
  
a 000  
b 10  
c 01000  
d 011  
e 001  
f 0101  
g 01001  
h 11  
...then the output file ought to be called tomato.tbb and contain those bytes (those of you familiar with binary and hex will be able to do the conversion in their head, or at least with pencil and paper):  
  
61 10 00  
62 a0 00  
63 44 00  
64 70 00  
65 30 00  
66 58 00  
67 4c 00  
68 e0 00                       

In [67]:
in_file = open("Micro0a.tbt", "r")
out_file = open("Micro0a.tbb","w")

for line in in_file:
    line=line.strip('\n')
    symbol = line.split(' ',1)[0]
    image = line.split(' ',1)[1] + '1'
    if len(image) > 16:
        raise Exception("Image is longer than 15 bits!")
    while len(image) < 16:
        image +='0'
        
    # a single ASCII character
    if len(symbol) == 1:
        out_file.write(hex(ord(symbol))[2:]+' '+hex(int(image[0:4],2))[2:]+hex(int(image[4:8],2))[2:]+
                       ' '+hex(int(image[8:12],2))[2:]+hex(int(image[12:16],2))[2:]+"\n")
    # a two-hex-digit representation
    else:
        out_file.write(symbol+' '+hex(int(image[0:4],2))[2:]+hex(int(image[4:8],2))[2:]+
                       ' '+hex(int(image[8:12],2))[2:]+hex(int(image[12:16],2))[2:]+"\n")
        
in_file.close()
out_file.close()

### (b)
Write a tbb2tbt program that converts from the binary table representation to the text representation. If you give it tomato.tbb it must give back tomato.tbt, as above.

In [66]:
in_file = open("Micro0b.tbb", "r")
out_file = open("Micro0b.tbt","w")

for line in in_file:
    line=line.strip('\n')
    symbol = line.split(' ',2)[0]
    image1 = bin(int(line.split(' ',2)[1][0],16))[2:].zfill(4) + bin(int(line.split(' ',2)[1][1],16))[2:].zfill(4)
    image2 = bin(int(line.split(' ',2)[2][0],16))[2:].zfill(4) + bin(int(line.split(' ',2)[2][1],16))[2:].zfill(4)
    image = image1 + image2
    out_file.write(chr(int(symbol,16)) + " " + image.rstrip('0')[:-1] + "\n")
    
in_file.close()
out_file.close()